In [1]:
import sys
import numpy as np
import pandas as pd
import cv2
from pycocotools.coco import COCO
import imgaug as ia
import imgaug.augmenters as iaa
from imgaug.augmentables import Keypoint, KeypointsOnImage, BoundingBox, BoundingBoxesOnImage
from PIL import Image

class Process(object):
    numKeypoint = 17
    keypointsName = ['nose', 'left_eye', 'right_eye', 'left_ear', 'right_ear', 
                    'left_shoulder', 'right_shoulder', 'left_elbow', 'right_elbow', 
                    'left_wrist', 'right_wrist', 'left_hip', 'right_hip', 'left_knee', 
                    'right_knee', 'left_ankle', 'right_ankle']

    skeleton = [[15, 13], [13, 11], [16, 14], 
                [14, 12], [11, 12], [5, 11], 
                [6, 12], [5, 6], [5, 7], [6, 8], 
                [7, 9], [8, 10], [1, 2], [0, 1], 
                [0, 2], [1, 3], [2, 4], [3, 5], [4, 6]]
    
    trainAnnoPath = 'COCO/annotations/person_keypoints_train2017.json'
    valAnnoPath = 'COCO/annotations/person_keypoints_val2017.json'
    
    def loadData(self, setType):
        if setType == 'train':
            coco = COCO(self.trainAnnoPath)
        elif setType == 'val':
            coco = COCO(self.valAnnoPath)
            
        Data = []
        for aid in coco.anns.keys():
            ann =  coco.anns[aid]
            keypoints = ann['keypoints']
            
            if  keypoints.count(0) == 0: 
                imageName = setType + '2017/' + coco.imgs[ann['image_id']]['file_name']

                if (ann['image_id'] not in coco.imgs) or ann['iscrowd'] or (np.sum(keypoints[2::3]) == 0) or (ann['num_keypoints'] == 0):
                    continue
                data = dict(image_id = ann['image_id'], imgpath = imageName, bbox=ann['bbox'], keypoints=keypoints)
                Data.append(data)
            else:
                continue
            
        return Data
    
    def seperateData(self, data):
        imageIds = []
        images = []
        bbox = [] 
        keypoints = [] 
        visibility = []
        i = 0 
        for i in range(len(data)): 
            imageIds.append(data[i]['image_id'])
            img = cv2.imread('COCO/' + data[i]['imgpath'])
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            images.append(img)
            bbox.append(np.array(data[i]['bbox']).astype(int))
            
            kps = np.array(data[i]['keypoints']).astype(int)
            kps = kps.reshape(17,3)
            kps = self.reshapeKeypoints(kps)
            keypoints.append(kps['kps'])
            visibility.append(kps['visibility'])
                          
        return imageIds, np.array(images), bbox, keypoints, visibility
    
    def reshapeKeypoints(self, keypoints):
        keypoint = []
        visibility = []

        for kps in keypoints: 
            x,y,v = kps 
            keypoint.append(Keypoint(x,y))
            visibility.append(v)
            
        res = dict()
        res['kps'] = keypoint; res['visibility'] = visibility
        return res
        
    def augmentOnce(self, image, kps, bbox, imageId):
        imgh = image.shape[0]
        imgw = image.shape[1]
        
        x,y,w,h = bbox
        top = y
        left = x
        bottom = imgh - (y+h) 
        right = imgw - (x+w)
        
        seq =  iaa.Sequential([
            iaa.Crop(px=(top,right, bottom, left)),
            iaa.Resize(640)  
        ])
        
        imgAug, kpsAug = seq(image=image, keypoints=KeypointsOnImage(kps,shape=image.shape))
        
        img = cv2.cvtColor(imgAug, cv2.COLOR_BGR2RGB)
        newId = str(imageId).rjust(12, '0')
        cv2.imwrite('test/'+ newId + '.jpg', img)
        
        return imgAug, kpsAug
        
    def augmentationOnAll(self, images, kps, bbox, imageId):
        imgAugList = []
        kpsAugList = []
        
        i = 0 
        for i in range(len(images)):
            imgAug, kpsAug = self.augmentOnce(images[i], kps[i], bbox[i], imageId[i])
            imgAugList.append(imgAug)
            kpsAugList.append(kpsAug)
            
        return np.array(imgAugList), kpsAugList

    def mergeKpsVisibility(self, augKps, visibility, kpShape, MergeShape):
        kpsList = [] 
        i = 0
        for i in range(len(augKps)):
            for keypoint in augKps[i]:
                x,y = keypoint.x, keypoint.y
                kpsList.append(x)
                kpsList.append(y)
        kpsList = np.array(kpsList).reshape(kpShape)
        merge = np.dstack((kpsList, visibility))
        merge = merge.reshape(MergeShape)
        return merge
    
    def formatToDataframe(self, anno, keypoints):
        col_name  = ['nose_x', 'nose_y', 'nose_v', 'left_eye_x','left_eye_y', 'left_eye_v', 
                     'right_eye_x', 'right_eye_y', 'right_eye_v', 'left_ear_x', 'left_ear_y', 'left_ear_v',
                     'right_ear_x', 'right_ear_y', 'right_ear_v', 'left_shoulder_x', 'left_shoulder_y', 'left_shoulder_v',
                     'right_shoulder_x', 'right_shoulder_y', 'right_shoulder_v', 'left_elbow_x', 'left_elbow_y', 'left_elbow_v',
                     'right_elbow_x', 'right_elbow_y', 'right_elbow_v', 'left_wrist_x', 'left_wrist_y', 'left_wrist_v',
                     'right_wrist_x', 'right_wrist_y', 'right_wrist_v', 'left_hip_x', 'left_hip_y', 'left_hip_v', 
                     'right_hip_x', 'right_hip_y', 'right_hip_v', 'left_knee_x', 'left_knee_y', 'left_knee_v',
                    'right_knee_x', 'right_knee_y', 'right_knee_v', 'left_ankle_x', 'left_ankle_y', 'left_ankle_v', 
                     'right_ankle_x', 'right_ankle_y', 'right_ankle_v', 'image_path'  
                    ]
        data = []
        
        i = 0
        for i in range(len(keypoints)):
            cur_row = []
            kps = np.array(keypoints[i]).astype(int)
            for key in kps:
                cur_row.append(key)
            cur_row.append('COCO/aug-' + anno[i]['imgpath'])
            data.append(cur_row)
        
        return pd.DataFrame(data=data, columns=col_name)

processing = Process()

In [2]:
from sklearn.model_selection import train_test_split as tts
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

class Setup(object):
    setType = ''
    batchsize = 0
    fetchedData = []
    
    def __init__ (self, setType):
        self.setType = setType
        
    def getTrainingSet(self):
        batch = processing.loadData('train')
        
        imageId, images, bboxes, keypoints, visibility = processing.seperateData(batch)
           
        x, kps = processing.augmentationOnAll(images, keypoints, bboxes, imageId)
        
        Y = processing.mergeKpsVisibility(kps, visibility, (len(x), 17,2), (len(kps), 51))
        images = [] 
        
        return Y, batch

In [3]:
setup = Setup('train')

In [4]:
Y, batch = setup.getTrainingSet()

loading annotations into memory...
Done (t=10.89s)
creating index...
index created!


In [5]:
df = processing.formatToDataframe(batch, Y)
df

,nose_x,nose_y,nose_v,left_eye_x,left_eye_y,left_eye_v,right_eye_x,right_eye_y,right_eye_v,left_ear_x,...,right_knee_x,right_knee_y,right_knee_v,left_ankle_x,left_ankle_y,left_ankle_v,right_ankle_x,right_ankle_y,right_ankle_v,image_path
0,352,80,2,380,58,2,306,64,2,463,...,148,356,2,454,587,2,204,458,2,COCO/aug-train2017/000000528962.jpg
1,305,77,2,334,61,2,265,61,2,384,...,216,443,2,413,601,2,187,597,2,COCO/aug-train2017/000000085316.jpg
2,350,65,2,380,56,2,310,53,2,450,...,280,446,1,400,571,2,120,577,2,COCO/aug-train2017/000000057703.jpg
3,372,63,2,426,49,2,337,47,2,481,...,208,337,2,332,687,1,109,515,2,COCO/aug-train2017/000000515289.jpg
4,354,137,2,381,113,2,313,109,2,428,...,219,704,1,406,851,1,227,884,1,COCO/aug-train2017/000000022816.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8461,312,47,2,334,38,2,290,41,2,378,...,276,459,2,567,586,1,276,607,2,COCO/aug-train2017/000000264499.jpg
8462,429,50,2,461,37,2,405,33,2,494,...,194,450,2,356,579,2,226,507,2,COCO/aug-train2017/000000258364.jpg
8463,303,84,2,336,75,2,276,73,2,390,...,222,504,2,330,637,2,296,621,2,COCO/aug-train2017/000000013283.jpg
8464,400,87,2,440,73,2,380,69,2,510,...,330,405,2,600,592,2,410,577,2,COCO/aug-train2017/000000357770.jpg


In [6]:
df.to_csv('COCO_640.csv', index=False)